# Fast Train - Sentinel YOLOv5
Entrenamiento rapido de modelo custom para Seguridad Industrial.

In [5]:
# 1) Setup YOLOv5
from pathlib import Path
import os

ROOT = Path('/content') if Path('/content').exists() else Path.cwd()
YOLO_DIR = ROOT / 'yolov5'

if not YOLO_DIR.exists():
    !git clone https://github.com/ultralytics/yolov5 {YOLO_DIR}

%cd {YOLO_DIR}
!pip install -qr requirements.txt
!python -m pip install --upgrade pip

c:\content\yolov5


In [6]:
# 2) Dataset: unzip dataset_entrega.zip (Colab or local)
from pathlib import Path
import zipfile

candidate_zips = [
    Path('/content/dataset_entrega.zip'),
    Path('/content/drive/MyDrive/dataset_entrega.zip'),
    Path.cwd() / 'dataset_entrega.zip',
    Path.cwd().parent / 'dataset_entrega.zip',
]

zip_path = next((p for p in candidate_zips if p.exists()), None)
assert zip_path is not None, f"Zip not found. Checked: {[str(p) for p in candidate_zips]}"

extract_dir = Path('/content/dataset_entrega') if Path('/content').exists() else (Path.cwd() / 'dataset_entrega_extracted')
extract_dir.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(extract_dir)

print(f'Zip used: {zip_path}')
print(f'Extracted to: {extract_dir}')
!ls -la {extract_dir}
!find {extract_dir} -maxdepth 3 -type d

AssertionError: Zip not found. Checked: ['\\content\\dataset_entrega.zip', '\\content\\drive\\MyDrive\\dataset_entrega.zip', 'c:\\content\\yolov5\\dataset_entrega.zip', 'c:\\content\\dataset_entrega.zip']

In [ ]:
# 3) Auto-detect dataset root and create sentinel.yaml
from pathlib import Path
import yaml

base = Path('/content/dataset_entrega')
candidates = [base] + [p for p in base.iterdir() if p.is_dir()]
dataset_root = None

for c in candidates:
    if (c / 'images' / 'train').exists() and (c / 'images' / 'val').exists():
        dataset_root = c
        break

assert dataset_root is not None, 'Could not find images/train and images/val in extracted dataset.'

data_yaml = {
    'path': str(dataset_root),
    'train': 'images/train',
    'val': 'images/val',
    'nc': 4,
    'names': ['Casco', 'Chaleco', 'Persona_Sin_EPI', 'Peligro']
}

yaml_path = Path('/content/yolov5/data/sentinel.yaml')
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False, allow_unicode=True)

print('Dataset root:', dataset_root)
print('YAML written to:', yaml_path)
print(yaml_path.read_text(encoding='utf-8'))

In [ ]:
# 4) Train YOLOv5s for 50 epochs
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 50 --data data/sentinel.yaml --weights yolov5s.pt --name sentinel_demo

In [ ]:
# 5) best.pt location
from pathlib import Path

best_pt = Path('/content/yolov5/runs/train/sentinel_demo/weights/best.pt')
print('best.pt expected at:', best_pt)
print('Exists:', best_pt.exists())

# Also show latest generic exp path if name changes
all_best = sorted(Path('/content/yolov5/runs/train').glob('*/weights/best.pt'))
print('Found best.pt files:')
for p in all_best:
    print(' -', p)

In [ ]:
# 6) Optional direct download from Colab
# If this fails in VS Code kernel, use file explorer download instructions below.
try:
    from google.colab import files
    files.download(str(best_pt))
except Exception as e:
    print('Direct browser download not available in this environment:', e)